In [1]:
import sys
sys.path.append("../module/")
sys.path.append("../learning/")

import pickle
import menconn
from typelinking import *
import time
import marisa_trie
import pickle
from dataset import *
from wikidata_linker_utils.offset_array import OffsetArray
import train_type as tp
from collections import defaultdict
from tqdm import tqdm
import re
from functools import partial
from tqdm import tqdm_notebook
import os

../module/typelinking.py:112: SyntaxWarning: name 'dataroot' is assigned to before global declaration
  global dataroot
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with open("ja_test_data.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
results = []


INFO:tensorflow:Restoring parameters from ../ja_model/model.ckpt


In [4]:
for d in tqdm_notebook(data):
    if len(d[2]) == 0:
        continue
    sentence = d[1]
    ts = [str(t[0]) for t in d[2]]
    true_entities = [str(t[1]).replace('_', ' ') for t in d[2]]
    tokenize = partial(menconn.ja_tokenize, ts=ts)
    sent_splits, model_probs = solve_model_probs(sentence, tagger, tokenize=tokenize)
    entities = run(ts, sent_splits, model_probs, indices2title, type_oracle, trie, trie_index2indices_values, trie_index2indices_counts)
    preds = []
    for entity in entities:
        if entity is not None:
            preds.append(entity['ja'])
        else:
            preds.append(None)
    results += [{'doc_id':d[0], 'mention':x, 'true': y, 'pred': z} for x,y,z in zip(ts, true_entities, preds)]

HBox(children=(IntProgress(value=0, max=340), HTML(value='')))

In [5]:
import pandas as pd
df = pd.DataFrame(results)

In [6]:
df.to_csv("ja_results.csv", index=False)

In [7]:
matched = df['pred'] == df['true']
length = df['pred'].shape[0]
assert len(df['pred']) == len(df['true'])

accuracy = float(sum(matched))/float(length)
accuracy

0.7755237227788359

In [8]:
from sklearn.metrics import f1_score
true = df['true'] == df['true']
pred = matched
f1_score(true, pred)

0.8735717949913725

In [9]:
df

,doc_id,mention,pred,true
0,PN1a_00002,大津西小学校,None,鳴門市立大津西小学校
1,PN1a_00002,教授,教授,教授
2,PN1a_00002,教頭,教頭,教頭
3,PN1a_00002,日本大学,日本大学,日本大学
4,PN1a_00002,草加市,草加市,草加市
5,PN1a_00002,三重県,三重県,三重県
6,PN1a_00002,欧米,欧米,欧米
7,PN1a_00002,埼玉県,埼玉県,埼玉県
8,PN1a_00002,マヨネーズ,マヨネーズ,マヨネーズ
9,PN1a_00002,豊島区,豊島区,豊島区
